<a href="https://colab.research.google.com/github/Naveen131/Machine-learning/blob/master/recommend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
import turicreate as tc

In [4]:
from google.colab import files
uploaded = files.upload()

Saving user_tran.csv to user_tran.csv
Saving customers.csv to customers.csv


# Load Data

In [0]:
customers = pd.read_csv('customers.csv') # list of 1000 customer ids
transactions = pd.read_csv('user_tran.csv') # user transactions

In [25]:
customers.head()

,customerId
0,1553
1,20400
2,19750
3,6334
4,27773


In [26]:
transactions.head()

,customerId,products
0,0,20
1,1,2|2|23|68|68|111|29|86|107|152
2,2,111|107|29|11|11|11|33|23
3,3,164|227
4,5,2|2


# Data Preparation

In [0]:

data = pd.melt(transactions.set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products')


In [32]:
data= data.groupby((['customerId', 'products'])).agg({'products': 'count'})

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: 'products' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version
  """Entry point for launching an IPython kernel.


In [33]:
data.head(50)

products
customerId products          
0          1                9
1          1                9
2          1                1
3          1                5
           2                1
4          1                6
5          1                7
6          1                4
7          1                2
8          1               19
9          1               10
10         1               14
           2                1
11         1               14
12         1                2
13         1                4
14         1               18
15         1                2
16         1                5
17         1                8
18         1                1
19         1                6
20         1                5
21         1                9
           2                1
22         1               14
23         1                5
24         1                8
25         1               11
           2                1
27         1                6
           2                1
28         1                1
29         1                8
30         1                6
31         1                4
32         1                5
33         1               10
           2                1
34         1                5
35         1                8
           2                1
36         1                4
37         1                4
38         1               17
39         1               13
40         1                9
           2                2
41         1                3
42         1                1

In [0]:
data = data.rename(columns={'products': 'purchase_count'})

In [0]:
data = data.reset_index()

In [0]:
data = data.rename(columns={'products': 'productsId'})

In [37]:
data.head()

,customerId,productsId,purchase_count
0,0,1,9
1,1,1,9
2,2,1,1
3,3,1,5
4,3,2,1


In [0]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy
data_dummy = create_data_dummy(data)

In [0]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productsId')

In [40]:
df_matrix.head()

productsId,1,2,3,4,5,6,7,8,9,10,15
customerId,,,,,,,,,,,
0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())

In [42]:
df_matrix_norm.head()

productsId,1,2,3,4,5,6,7,8,9,10,15
customerId,,,,,,,,,,,
0,0.173913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.173913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.086957,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.108696,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
d = df_matrix_norm.reset_index() 
d.index.names = ['scaled_purchase_freq'] 
data_norm = pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm.head()

(25155, 3)


,customerId,productsId,scaled_purchase_freq
0,0,1,0.173913
1,1,1,0.173913
2,2,1,0.000000
3,3,1,0.086957
4,4,1,0.108696


In [0]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()

In [0]:
def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [0]:
user_id = 'customerId'
item_id = 'productsId'
users_to_recommend = list(customers[user_id])
n_rec = 10 # number of items to recommend
n_display = 30 # to display the first few rows in an output dataset

In [0]:
def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
         model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [0]:
train_data, test_data = split_data(data)
train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

# Collaborative Filtering

In [56]:
name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 20165 observations with 19645 users and 9 items.

Data prepared in: 0.042664s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

+------------+------------+-----------------------+------+
| customerId | productsId |         score         | rank |
+------------+------------+-----------------------+------+
|    1553    |     2      |   1.0114215016365051  |  1   |
|    1553    |     3      |   0.5018110871315002  |  2   |
|    1553    |     4      |   0.2777579426765442  |  3   |
|    1553    |     9      |  0.18263643980026245  |  4   |
|    1553    |     7      |  0.14730334281921387  |  5   |
|    1553    |     6      |  0.10030156373977661  |  6   |
|    1553    |     8      |  0.08729088306427002  |  7   |
|    1553    |     5      |  0.061724066734313965 |  8   |
|   20400    |     2      |  0.03778516583972507  |  1   |
|   20400    |     3      |  0.019596126344468858 |  2   |
|   20400    |     4      |  0.010851144790649414 |  3   |
|   20400    |     9      |  0.006967756483289931 |  4   |
|   20400    |     7      |  0.005704853269788954 |  5   |
|   20400    |     6      |  0.004772027333577474 |  6  

Generating candidate set for working with new users.

Finished training in 1.01168s

recommendations finished on 1000/1000 queries. users per second: 41133.6

In [57]:
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 20165 observations with 19669 users and 10 items.

Data prepared in: 0.04376s

...

Finalizing lookup tables.

+------------+------------+----------------------+------+
| customerId | productsId |        score         | rank |
+------------+------------+----------------------+------+
|    1553    |     2      | 0.12205260992050171  |  1   |
|    1553    |     3      | 0.04250669479370117  |  2   |
|    1553    |     4      | 0.03032892942428589  |  3   |
|    1553    |     5      | 0.011697709560394287 |  4   |
|    1553    |     7      | 0.010130524635314941 |  5   |
|    1553    |     8      | 0.010130524635314941 |  6   |
|    1553    |     9      | 0.010130524635314941 |  7   |
|    1553    |     15     | 0.007163345813751221 |  8   |
|    1553    |     10     | 0.007163345813751221 |  9   |
|   20400    |     2      | 0.12205260992050171  |  1   |
|   20400    |     3      | 0.04250669479370117  |  2   |
|   20400    |     4      | 0.03032892942428589  |  3   |
|   20400    |     5      | 0.011697709560394287 |  4   |
|   20400    |     7      | 0.010130524635314941 |  5   |
|   20400    |

Generating candidate set for working with new users.

Finished training in 0.030737s

recommendations finished on 1000/1000 queries. users per second: 149254

In [58]:
name = 'cosine' 
target = 'scaled_purchase_freq' 
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 20124 observations with 19645 users and 3 items.

Data prepared in: 0.043052s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.009624s

recommendations finished on 1000/1000 queries. users per second: 155135

In [59]:
name = 'pearson'
target = 'purchase_count'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 20165 observations with 19645 users and 9 items.

Data prepared in: 0.048436s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.010753s

recommendations finished on 1000/1000 queries. users per second: 168237

+------------+------------+--------------------+------+
| customerId | productsId |       score        | rank |
+------------+------------+--------------------+------+
|    1553    |     2      | 1.2193853474154919 |  1   |
|    1553    |     3      | 1.0605849997321173 |  2   |
|    1553    |     4      |        1.0         |  3   |
|    1553    |     8      |        1.0         |  4   |
|    1553    |     9      |        1.0         |  5   |
|    1553    |     6      |        1.0         |  6   |
|    1553    |     5      |        1.0         |  7   |
|    1553    |     7      |        1.0         |  8   |
|   20400    |     1      | 2.4718756526711205 |  1   |
|   20400    |     2      | 1.1128742181421831 |  2   |
|   20400    |     3      | 1.0395923498060375 |  3   |
|   20400    |     4      |        1.0         |  4   |
|   20400    |     8      |        1.0         |  5   |
|   20400    |     9      |        1.0         |  6   |
|   20400    |     6      |        1.0         |

In [60]:
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)


Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 20165 observations with 19669 users and 10 items.

Data prepared in: 0.043652s

...

Finalizing lookup tables.

+------------+------------+-------+------+
| customerId | productsId | score | rank |
+------------+------------+-------+------+
|    1553    |     4      |  0.0  |  1   |
|    1553    |     2      |  0.0  |  2   |
|    1553    |     7      |  0.0  |  3   |
|    1553    |     3      |  0.0  |  4   |
|    1553    |     15     |  0.0  |  5   |
|    1553    |     5      |  0.0  |  6   |
|    1553    |     10     |  0.0  |  7   |
|    1553    |     8      |  0.0  |  8   |
|    1553    |     9      |  0.0  |  9   |
|   20400    |     4      |  0.0  |  1   |
|   20400    |     2      |  0.0  |  2   |
|   20400    |     7      |  0.0  |  3   |
|   20400    |     3      |  0.0  |  4   |
|   20400    |     15     |  0.0  |  5   |
|   20400    |     5      |  0.0  |  6   |
|   20400    |     10     |  0.0  |  7   |
|   20400    |     8      |  0.0  |  8   |
|   20400    |     9      |  0.0  |  9   |
|   19750    |     4      |  0.0  |  1   |
|   19750    |     2      |  0.0  |  2   |
|   19750  

Generating candidate set for working with new users.

Finished training in 0.010373s

recommendations finished on 1000/1000 queries. users per second: 78727.8

In [61]:
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 20124 observations with 19645 users and 3 items.

Data prepared in: 0.039709s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.03524s

recommendations finished on 1000/1000 queries. users per second: 209644

+------------+------------+----------------------+------+
| customerId | productsId |        score         | rank |
+------------+------------+----------------------+------+
|    1553    |     3      | 0.07175218799839851  |  1   |
|    1553    |     2      |  0.0451073413803464  |  2   |
|    1553    |     1      | 0.030577137063642162 |  3   |
|   20400    |     3      | 0.07175218799839851  |  1   |
|   20400    |     2      |  0.0451073413803464  |  2   |
|   20400    |     1      | 0.030577137063642162 |  3   |
|   19750    |     3      | 0.07167287408441739  |  1   |
|   19750    |     2      | 0.04261825141452609  |  2   |
|    6334    |     3      | 0.07167287408441739  |  1   |
|    6334    |     2      | 0.04261825141452609  |  2   |
|   27773    |     3      | 0.07167287408441739  |  1   |
|   27773    |     2      | 0.04261825141452609  |  2   |
|   20027    |     3      | 0.07175218799839851  |  1   |
|   20027    |     2      |  0.0451073413803464  |  2   |
|   20027    |

In [63]:
name = 'popularity'
target = 'purchase_count'
popularity = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)


Preparing data set.

Data has 20165 observations with 19645 users and 9 items.

Data prepared in: 0.045249s

20165 observations to process; with 9 unique items.

recommendations finished on 1000/1000 queries. users per second: 314861

+------------+------------+--------------------+------+
| customerId | productsId |       score        | rank |
+------------+------------+--------------------+------+
|    1553    |     2      | 1.1391752577319587 |  1   |
|    1553    |     3      | 1.044776119402985  |  2   |
|    1553    |     4      |        1.0         |  3   |
|    1553    |     8      |        1.0         |  4   |
|    1553    |     9      |        1.0         |  5   |
|    1553    |     6      |        1.0         |  6   |
|    1553    |     5      |        1.0         |  7   |
|    1553    |     7      |        1.0         |  8   |
|   20400    |     1      | 2.4938138508136967 |  1   |
|   20400    |     2      | 1.1391752577319587 |  2   |
|   20400    |     3      | 1.044776119402985  |  3   |
|   20400    |     4      |        1.0         |  4   |
|   20400    |     8      |        1.0         |  5   |
|   20400    |     9      |        1.0         |  6   |
|   20400    |     6      |        1.0         |

In [64]:
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 20165 observations with 19669 users and 10 items.

Data prepared in: 0.038944s

20165 observations to process; with 10 unique items.

recommendations finished on 1000/1000 queries. users per second: 204374

+------------+------------+-------+------+
| customerId | productsId | score | rank |
+------------+------------+-------+------+
|    1553    |     4      |  1.0  |  1   |
|    1553    |     2      |  1.0  |  2   |
|    1553    |     7      |  1.0  |  3   |
|    1553    |     3      |  1.0  |  4   |
|    1553    |     15     |  1.0  |  5   |
|    1553    |     5      |  1.0  |  6   |
|    1553    |     10     |  1.0  |  7   |
|    1553    |     8      |  1.0  |  8   |
|    1553    |     9      |  1.0  |  9   |
|   20400    |     4      |  1.0  |  1   |
|   20400    |     2      |  1.0  |  2   |
|   20400    |     7      |  1.0  |  3   |
|   20400    |     3      |  1.0  |  4   |
|   20400    |     15     |  1.0  |  5   |
|   20400    |     5      |  1.0  |  6   |
|   20400    |     10     |  1.0  |  7   |
|   20400    |     8      |  1.0  |  8   |
|   20400    |     9      |  1.0  |  9   |
|   19750    |     4      |  1.0  |  1   |
|   19750    |     2      |  1.0  |  2   |
|   19750  

In [65]:
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 20124 observations with 19645 users and 3 items.

Data prepared in: 0.043773s

20124 observations to process; with 3 unique items.

recommendations finished on 1000/1000 queries. users per second: 340252

+------------+------------+----------------------+------+
| customerId | productsId |        score         | rank |
+------------+------------+----------------------+------+
|    1553    |     3      | 0.07246376811594203  |  1   |
|    1553    |     2      | 0.047023809523809496 |  2   |
|    1553    |     1      | 0.03230928777724797  |  3   |
|   20400    |     3      | 0.07246376811594203  |  1   |
|   20400    |     2      | 0.047023809523809496 |  2   |
|   20400    |     1      | 0.03230928777724797  |  3   |
|   19750    |     3      | 0.07246376811594203  |  1   |
|   19750    |     2      | 0.047023809523809496 |  2   |
|    6334    |     3      | 0.07246376811594203  |  1   |
|    6334    |     2      | 0.047023809523809496 |  2   |
|   27773    |     3      | 0.07246376811594203  |  1   |
|   27773    |     2      | 0.047023809523809496 |  2   |
|   20027    |     3      | 0.07246376811594203  |  1   |
|   20027    |     2      | 0.047023809523809496 |  2   |
|   20027    |

# Model Evaluation

In [0]:
models_w_counts = [popularity, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]
names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']


In [76]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

PROGRESS: Evaluate model Popularity Model on Purchase Counts


recommendations finished on 1000/5014 queries. users per second: 169405

recommendations finished on 2000/5014 queries. users per second: 200300

recommendations finished on 3000/5014 queries. users per second: 182482

recommendations finished on 4000/5014 queries. users per second: 189816

...

recommendations finished on 1000/4999 queries. users per second: 351247

recommendations finished on 2000/4999 queries. users per second: 339156

recommendations finished on 3000/4999 queries. users per second: 304538

recommendations finished on 4000/4999 queries. users per second: 294074


Precision and recall summary statistics by cutoff
+--------+----------------------+----------------------+
| cutoff |    mean_precision    |     mean_recall      |
+--------+----------------------+----------------------+
|   1    | 0.007801560312062434 | 0.007201440288057603 |
|   2    | 0.028005601120224072 | 0.05071014202840547  |
|   3    | 0.34370207374808315  |         1.0          |
|   4    | 0.34370207374808315  |         1.0          |
|   5    |  0.3437020737480832  |         1.0          |
|   6    |  0.3437020737480832  |         1.0          |
|   7    |  0.3437020737480832  |         1.0          |
|   8    |  0.3437020737480832  |         1.0          |
|   9    |  0.3437020737480832  |         1.0          |
|   10   |  0.3437020737480832  |         1.0          |
+--------+----------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.055053928302984965

Per User RMSE (best)
+------------+---------------------+-------+
| customerId |         rm

In [78]:
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 20165 observations with 19669 users and 10 items.

Data prepared in: 0.054823s

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.011302s

recommendations finished on 1000/1000 queries. users per second: 58541.2

In [80]:
recom = cos_dummy.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

recommendations finished on 1000/1000 queries. users per second: 89750.5

+------------+------------+----------------------+------+
| customerId | productsId |        score         | rank |
+------------+------------+----------------------+------+
|    1553    |     2      | 0.12205260992050171  |  1   |
|    1553    |     3      | 0.04250669479370117  |  2   |
|    1553    |     4      | 0.03032892942428589  |  3   |
|    1553    |     5      | 0.011697709560394287 |  4   |
|    1553    |     7      | 0.010130524635314941 |  5   |
|    1553    |     8      | 0.010130524635314941 |  6   |
|    1553    |     9      | 0.010130524635314941 |  7   |
|    1553    |     15     | 0.007163345813751221 |  8   |
|    1553    |     10     | 0.007163345813751221 |  9   |
|   20400    |     2      | 0.12205260992050171  |  1   |
|   20400    |     3      | 0.04250669479370117  |  2   |
|   20400    |     4      | 0.03032892942428589  |  3   |
|   20400    |     5      | 0.011697709560394287 |  4   |
|   20400    |     7      | 0.010130524635314941 |  5   |
|   20400    |

In [0]:
def create_output(model, users_to_recommend, n_rec, print_csv=True):
    recomendation = model.recommend(users=users_to_recommend, k=n_rec)
    df_rec = recomendation.to_dataframe()
    df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))
    df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates() \
        .sort_values('customerId').set_index('customerId')
    if print_csv:
        df_output.to_csv('recommendation.csv')
        print("An output file can be found in 'output' folder with name 'option1_recommendation.csv'")
    return df_output

In [84]:
df_output = create_output(pear_norm, users_to_recommend, n_rec, print_csv=True)
print(df_output.shape)
df_output.head()

recommendations finished on 1000/1000 queries. users per second: 177148

An output file can be found in 'output' folder with name 'option1_recommendation.csv'
(1000, 1)


,recommendedProducts
customerId,
4,3|2
11,2|3
12,3|2
16,3|2|1
21,3|2|1
